In [1]:
import os
import random
import cv2
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
!mkdir ohaze/haze_syn

In [10]:
def create_dataset(img_dir, data_dir, patch_size=500):
    # img_dir: dir of haze-free images
    # patch_size: size of image patch

    img_path = os.listdir(img_dir)

    for image_name in img_path:
        fullname = os.path.join(img_dir, image_name)
        print(fullname)
        img = cv2.imread(fullname)

        w, h, _ = img.shape

        num_w = w // patch_size
        num_h = h // patch_size

        # Copy the original image
        hazy_img = img.copy()
        tmap = np.zeros((hazy_img.shape[0], hazy_img.shape[1]))

        for i in range(num_w):
            for j in range(num_h):
                t = random.random()
                tmap[i * patch_size : (i+1) * patch_size,
                    j * patch_size : (j+1) * patch_size] = t * 255

        # Handle the remainder (right and bottom strips)
        for i in range(num_w + 1):
            t = random.random()
            tmap[i * patch_size : (i+1) * patch_size, num_h * patch_size :] = t * 255

        for j in range(num_h + 1):
            t = random.random()
            tmap[num_w * patch_size :, j * patch_size : (j+1) * patch_size] = t * 255

        img = img.astype(np.float32) / 255
        tmap = tmap.astype(np.float32) / 255

        # Create the guided filter. 
        guided_filter = cv2.ximgproc.createGuidedFilter(guide=img, radius=270, eps=0.001)
        refined_tmap = (255 * guided_filter.filter(tmap)).astype(np.uint8)

        # Write refined tmap to disk
        tmap_img_name = os.path.join(data_dir, image_name.replace('GT','tmap'))
        cv2.imwrite(tmap_img_name, refined_tmap)

        # Generate hazy image using atmospheric scattering model with atmlight = 255
        refined_tmap = refined_tmap[:,:,np.newaxis]
        hazy_img = np.clip(img * refined_tmap + (255 - refined_tmap), 0, 255).astype('uint8')

        # Write the hazy image to disk
        hazy_img_name = os.path.join(data_dir, image_name.replace('GT','syn'))
        cv2.imwrite(hazy_img_name, hazy_img)


In [12]:
create_dataset("ohaze/GT/", "ohaze/haze_syn/")

ohaze/GT/30_outdoor_GT.jpg
ohaze/GT/12_outdoor_GT.jpg
ohaze/GT/37_outdoor_GT.jpg
ohaze/GT/29_outdoor_GT.jpg
ohaze/GT/15_outdoor_GT.jpg
ohaze/GT/02_outdoor_GT.jpg
ohaze/GT/41_outdoor_GT.jpg
ohaze/GT/20_outdoor_GT.jpg
ohaze/GT/05_outdoor_GT.jpg
ohaze/GT/27_outdoor_GT.jpg
ohaze/GT/39_outdoor_GT.jpg
ohaze/GT/35_outdoor_GT.jpg
ohaze/GT/03_outdoor_GT.JPG
ohaze/GT/09_outdoor_GT.jpg
ohaze/GT/17_outdoor_GT.jpg
ohaze/GT/21_outdoor_GT.JPG
ohaze/GT/32_outdoor_GT.jpg
ohaze/GT/10_outdoor_GT.jpg
ohaze/GT/44_outdoor_GT.jpg
ohaze/GT/19_outdoor_GT.jpg
ohaze/GT/07_outdoor_GT.jpg
ohaze/GT/25_outdoor_GT.jpg
ohaze/GT/43_outdoor_GT.jpg
ohaze/GT/22_outdoor_GT.jpg
ohaze/GT/45_outdoor_GT.jpg
ohaze/GT/18_outdoor_GT.jpg
ohaze/GT/06_outdoor_GT.jpg
ohaze/GT/24_outdoor_GT.jpg
ohaze/GT/01_outdoor_GT.jpg
ohaze/GT/42_outdoor_GT.jpg
ohaze/GT/23_outdoor_GT.jpg
ohaze/GT/34_outdoor_GT.jpg
ohaze/GT/08_outdoor_GT.jpg
ohaze/GT/16_outdoor_GT.jpg
ohaze/GT/33_outdoor_GT.jpg
ohaze/GT/11_outdoor_GT.jpg
ohaze/GT/40_outdoor_GT.jpg
o